In [4]:
from openai import OpenAI
import re

# OpenAI API 설정
client = OpenAI(api_key="sk...")  

# 감정 라벨 목록
emotion_labels = ["기쁨", "슬픔", "외로움", "두려움", "평온", "설렘", "신남", "분노"]

def generate_response(user_input):
    # GPT-4o로 대화 + 감정 분석 요청
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 독거노인분들과 대화해주는 친구야. 친절하게 대화를 이어가고, 마지막에는 반드시 감정 분석을 해서 '감정: [감정]' 형식으로 포함해야 해. 감정은 다음 중 하나로만 선택해: " + ", ".join(emotion_labels)},
            {"role": "user", "content": user_input}
        ],
        max_tokens=500,
        temperature=0.7
    )

    gpt_reply = response.choices[0].message.content

    # 감정 추출 (정규표현식 활용)
    emotion_match = re.search(r"감정:\s*(\w+)", gpt_reply)
    emotion = emotion_match.group(1) if emotion_match else "분석 실패"

    # 감정 부분 제거하고 깨끗한 대화 응답만 추출
    clean_reply = re.sub(r"감정:\s*\w+", "", gpt_reply).strip()

    return clean_reply, emotion

In [5]:
# 샘플 데이터
sample_data = [
    {"user_input": "태풍에 대한 기억은 옛날에 매일 민간 올 때 제가 울릉도 있었거든요. 울릉도 있었는데, 그 이틀은 우리가 나오지 못했어요. 그때 호텔에서 이렇게 내다보는데 얼마나 태풍이 불면 차가 막 뒤로 움직이더라고요."}
]

In [6]:
# 실행
for sample in sample_data:
    response_text, detected_emotion = generate_response(sample["user_input"])
    print(f"대화: {sample['user_input']}")
    print(f"답변: {response_text}")
    print(f"감정 분석 결과: {detected_emotion}")
    print("-" * 50)

질문: 태풍에 대한 기억은 옛날에 매일 민간 올 때 제가 울릉도 있었거든요. 울릉도 있었는데, 그 이틀은 우리가 나오지 못했어요. 그때 호텔에서 이렇게 내다보는데 얼마나 태풍이 불면 차가 막 뒤로 움직이더라고요.
답변: 울릉도에서 태풍을 경험하셨다니 정말 놀라웠겠어요. 자연의 힘이 얼마나 강력한지 직접 느끼셨을 것 같아요. 차가 뒤로 움직일 정도라니 그때 많이 무섭기도 했겠어요. 그래도 무사히 그 상황을 잘 넘기셔서 다행이에요. 혹시 그때 이후로 태풍이나 자연재해에 대해서 다른 생각이 들진 않으셨나요?
감정 분석 결과: 두려움
--------------------------------------------------
